# CS 445 Project Report: Nikhil-GPT, Machine Learning Chatbot

*by Daniel Ives and Kenny Hotra, May ??, 2023*

This document is just a suggestion on how to structure your final project report.  You may choose to structure your report differently.

At the end is a code cell that counts words for you.

Do not include any of the text in this document, except possibly for the section headings, in your project report.

## Introduction

What, why, very brief overview of methods and results.

I'm very interested in the grammar, or lack thereof, used in tweets.  I will try to automatically recognize presidential tweets by counting occurrences of the word "covfefe"....

## Methods

Steps I took.  Resources I used, such as code from the class, on-line resources, research articles, books [Goodfellow, et al., 2016], ....

REQUIRED: If this is a team project, clearly describe in detail what each team member did.

## Results

Show all results.  Intermediate results might be shown in above Methods section.  Plots, tables, whatever.

## Conclusions

What I learned.  What was difficult.  Changes I had to make to timeline.

### References

* ?

Your report for a single person team should contain **approximately** 2,000 words times number of team members, in markdown cells.  You can count words by running the following python code in your report directory.  Projects with two people, for example, should contain about 4,000 words, a four-person team should submit a report of approximately 8,000 words.

Of course, your results and analysis speak much more than the word count.  Deep analysis in a shorter form is better than vague, over-wordy non-analysis.

In [1]:
import io
from nbformat import current
import glob
nbfile = glob.glob('Ives-Hotra-Project-Report.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file Ives-Hotra-Project-Report.ipynb is 254
